In [51]:
import pandas as pd
import numpy as np

In [ ]:
# https://www.kaggle.com/competitions/digit-recognizer/

In [196]:
train_df = pd.read_csv("./digit-recognizer/train.csv")

In [197]:
test_df = pd.read_csv("./digit-recognizer/test.csv")

In [205]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [206]:
train_df, val_df = train_test_split(train_df)
train_X = torch.as_tensor(train_df.values)[:,1:]
train_y = torch.as_tensor(train_df.values)[:,0]
val_X = torch.as_tensor(val_df.values)[:,1:]
val_y = torch.as_tensor(val_df.values)[:,0]
test_X = torch.as_tensor(test_df.values)
print(train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape)

torch.Size([31500, 784]) torch.Size([31500]) torch.Size([10500, 784]) torch.Size([10500]) torch.Size([28000, 784])


In [207]:
train_dl = DataLoader(TensorDataset(train_X, train_y), batch_size=128, shuffle=True, num_workers=4)
val_dl = DataLoader(TensorDataset(val_X, val_y), batch_size=128, shuffle=False, num_workers=4)
test_dl = DataLoader(TensorDataset(test_X), batch_size=128, shuffle=False, num_workers=4)

In [249]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1, stride=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1, stride=1)
        self.conv3 = nn.Conv2d(64,128, 3, padding=1, stride=1)
        self.flatten = nn.Flatten(1,-1)
        self.fc1 = nn.Linear(128*3*3, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = x/255
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)

        return x


net = Net()


In [250]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3, betas=(0.9, 0.98))


In [251]:
train_loader = train_dl
val_loader = val_dl

In [252]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3, betas=(0.9, 0.95))
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # print(type(inputs), type(labels))
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        # print(type(outputs))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
        # break
    correct = 0
    total = 0

    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the validation images: {100 * correct / total} %')

Accuracy of the network on the validation images: 96.61904761904762 %
Accuracy of the network on the validation images: 98.15238095238095 %
Accuracy of the network on the validation images: 98.54285714285714 %
Accuracy of the network on the validation images: 98.6952380952381 %
Accuracy of the network on the validation images: 98.56190476190476 %
Accuracy of the network on the validation images: 98.9047619047619 %
Accuracy of the network on the validation images: 98.78095238095239 %
Accuracy of the network on the validation images: 98.35238095238095 %
Accuracy of the network on the validation images: 98.58095238095238 %
Accuracy of the network on the validation images: 98.77142857142857 %


In [224]:
test_dl = DataLoader(test_X, batch_size=128, shuffle=False, num_workers=4)

In [226]:
with torch.no_grad():
    predicted_set = []
    for data in test_dl:
        images = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        predicted_set.append(predicted)

In [227]:
predicted_set

[tensor([2, 0, 9, 9, 3, 7, 0, 3, 0, 3, 5, 7, 4, 0, 4, 3, 3, 1, 9, 0, 9, 1, 1, 5,
         7, 4, 2, 7, 4, 7, 7, 5, 4, 2, 6, 2, 5, 5, 1, 6, 7, 7, 4, 9, 8, 7, 8, 2,
         6, 7, 6, 8, 8, 3, 8, 2, 1, 2, 2, 0, 4, 1, 7, 0, 0, 0, 1, 9, 0, 1, 6, 5,
         8, 8, 2, 8, 9, 9, 2, 3, 5, 7, 1, 0, 9, 2, 4, 3, 6, 7, 2, 0, 6, 6, 1, 4,
         3, 9, 7, 4, 0, 9, 2, 0, 7, 3, 0, 5, 0, 8, 0, 0, 4, 7, 1, 7, 1, 1, 3, 3,
         3, 7, 2, 8, 6, 3, 8, 7]),
 tensor([7, 4, 3, 5, 6, 0, 0, 0, 3, 1, 3, 6, 4, 3, 4, 5, 5, 8, 7, 7, 2, 8, 4, 3,
         5, 6, 5, 3, 7, 5, 7, 8, 3, 0, 4, 5, 1, 3, 7, 6, 3, 0, 2, 7, 8, 6, 1, 3,
         7, 4, 1, 2, 4, 2, 5, 2, 4, 9, 2, 1, 6, 0, 6, 1, 4, 9, 6, 0, 9, 7, 6, 9,
         1, 9, 0, 9, 9, 0, 8, 4, 6, 2, 0, 9, 3, 6, 3, 2, 1, 6, 3, 4, 2, 3, 1, 2,
         2, 0, 4, 6, 1, 0, 0, 4, 9, 1, 7, 3, 2, 3, 8, 6, 8, 6, 2, 8, 5, 5, 4, 8,
         3, 5, 9, 7, 1, 3, 8, 4]),
 tensor([5, 1, 4, 5, 6, 3, 3, 5, 7, 0, 6, 8, 3, 1, 6, 0, 6, 3, 9, 3, 1, 5, 8, 4,
         0, 9, 2, 0, 5, 3, 7, 1, 9, 9, 

In [233]:
a = []
for i in predicted_set:
    for j in i:
      a.append(j.item())

In [234]:
a

[2,
 0,
 9,
 9,
 3,
 7,
 0,
 3,
 0,
 3,
 5,
 7,
 4,
 0,
 4,
 3,
 3,
 1,
 9,
 0,
 9,
 1,
 1,
 5,
 7,
 4,
 2,
 7,
 4,
 7,
 7,
 5,
 4,
 2,
 6,
 2,
 5,
 5,
 1,
 6,
 7,
 7,
 4,
 9,
 8,
 7,
 8,
 2,
 6,
 7,
 6,
 8,
 8,
 3,
 8,
 2,
 1,
 2,
 2,
 0,
 4,
 1,
 7,
 0,
 0,
 0,
 1,
 9,
 0,
 1,
 6,
 5,
 8,
 8,
 2,
 8,
 9,
 9,
 2,
 3,
 5,
 7,
 1,
 0,
 9,
 2,
 4,
 3,
 6,
 7,
 2,
 0,
 6,
 6,
 1,
 4,
 3,
 9,
 7,
 4,
 0,
 9,
 2,
 0,
 7,
 3,
 0,
 5,
 0,
 8,
 0,
 0,
 4,
 7,
 1,
 7,
 1,
 1,
 3,
 3,
 3,
 7,
 2,
 8,
 6,
 3,
 8,
 7,
 7,
 4,
 3,
 5,
 6,
 0,
 0,
 0,
 3,
 1,
 3,
 6,
 4,
 3,
 4,
 5,
 5,
 8,
 7,
 7,
 2,
 8,
 4,
 3,
 5,
 6,
 5,
 3,
 7,
 5,
 7,
 8,
 3,
 0,
 4,
 5,
 1,
 3,
 7,
 6,
 3,
 0,
 2,
 7,
 8,
 6,
 1,
 3,
 7,
 4,
 1,
 2,
 4,
 2,
 5,
 2,
 4,
 9,
 2,
 1,
 6,
 0,
 6,
 1,
 4,
 9,
 6,
 0,
 9,
 7,
 6,
 9,
 1,
 9,
 0,
 9,
 9,
 0,
 8,
 4,
 6,
 2,
 0,
 9,
 3,
 6,
 3,
 2,
 1,
 6,
 3,
 4,
 2,
 3,
 1,
 2,
 2,
 0,
 4,
 6,
 1,
 0,
 0,
 4,
 9,
 1,
 7,
 3,
 2,
 3,
 8,
 6,
 8,
 6,
 2,
 8,
 5,
 5,
 4,
 8,
 3,
 5,


In [235]:
len(a)

28000

In [236]:
b = range(1,28000+1)

In [238]:
b = np.arange(1,28000+1)

In [244]:
data = np.array([b,a]).transpose()

In [246]:
res = pd.DataFrame(data, columns=['ImageId', 'Label'])

In [248]:
res.to_csv('prediction.csv',index=False)